In [11]:
import random
import math
def is_prime(n):
    if n < 2:
        return False
    if n == 2:
        return True
    if n % 2 == 0:
        return False
    for i in range(3, int(math.sqrt(n)) + 1, 2):
        if n % i == 0:
            return False
    return True
def generate_key_pair(p, q):
    if not (is_prime(p) and is_prime(q)):
        raise ValueError("Both p and q must be prime.")
    n = p * q
    phi = (p - 1) * (q - 1)
    e = random.randrange(1, phi)
    while math.gcd(e, phi) != 1:
        e = random.randrange(1, phi)
    d = pow(e, -1, phi)
    return ((e, n), (d, n))

def encrypt(message, public_key):
    e, n = public_key
    encrypted_message = [pow(ord(c), e, n) for c in message]
    return encrypted_message

def decrypt(encrypted_message, private_key):
    d, n = private_key
    decrypted_message = [chr(pow(c, d, n)) for c in encrypted_message]
    return ''.join(decrypted_message)

# Example usage
p = 61
q = 53
public_key, private_key = generate_key_pair(p, q)
print(public_key)
print(private_key)
message = "My Name Is AreejIlyas 11447"
encrypted_message = encrypt(message, public_key)
decrypted_message = decrypt(encrypted_message, private_key)

print("Original message:", message)
print("Encrypted message:", encrypted_message)
print("Decrypted message:", decrypted_message)


(553, 3233)
(457, 3233)
Original message: My Name Is AreejIlyas 11447
Encrypted message: [1520, 731, 459, 645, 1712, 2000, 162, 459, 2099, 2263, 459, 2764, 2708, 162, 162, 371, 2099, 3158, 731, 1712, 2263, 459, 2537, 2537, 1589, 1589, 3052]
Decrypted message: My Name Is AreejIlyas 11447


In [2]:
import json
import ast
import socket
import threading
import numpy as np
import random
import math
import tkinter as tk
from tkinter import scrolledtext

class RSA:
    def __init__(self, p, q):
        self.p = p
        self.q = q
        self.public_key, self.private_key = self.generate_key_pair()

    def generate_key_pair(self):
        # Step 1: Compute n = p * q
        n = self.p * self.q

        # Step 2: Compute the totient of n, phi(n) = (p-1) * (q-1)
        phi = (self.p - 1) * (self.q - 1)

        # Step 3: Choose e such that 1 < e < phi and gcd(e, phi) = 1
        e = random.randrange(1, phi)
        while math.gcd(e, phi) != 1:
            e = random.randrange(1, phi)

        # Step 4: Compute d such that d * e ≡ 1 (mod phi)
        d = pow(e, -1, phi)

        # Return the public and private keys
        return ((e, n), (d, n))

    def encrypt(self, message):
        e, n = self.public_key
        encrypted_message = [pow(ord(c), e, n) for c in message]
        return encrypted_message

    def decrypt(self, encrypted_message):
        d, n = self.private_key
        decrypted_message = [chr(pow(c, d, n)) for c in encrypted_message]
        return ''.join(decrypted_message)


class Client:
    def __init__(self, host, port):
        self.host = host
        self.port = port
        self.client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.connect()

    def connect(self):
        self.client_socket.connect((self.host, self.port))
        print("Connected to server.")

    def send_message(self, message):
        self.client_socket.send(message.encode('utf-8'))

    def receive_message(self):
        data = self.client_socket.recv(1024).decode('utf-8')
        # Assuming you receive the JSON message in a variable called 'data'
        listData = json.loads(data)

        return listData


class App:
    def __init__(self, host, port):
        self.host = host
        self.port = port
        self.rsa = RSA(61, 53)
        self.client = Client(self.host, self.port)
        self.create_ui()

    def create_ui(self):
        self.window = tk.Tk()
        self.window.title("RSA Chat")
        self.input_text = tk.Entry(self.window, width=50)
        self.input_text.pack(pady=10)
        self.send_button = tk.Button(self.window, text="Send", command=self.send_message)
        self.send_button.pack(pady=5)
        self.output_text = scrolledtext.ScrolledText(self.window, width=60, height=10)
        self.output_text.pack(pady=10)
        self.receive_thread = threading.Thread(target=self.receive_messages)
        self.receive_thread.start()

    def send_message(self):
        message = self.input_text.get()
        encrypted_message = self.rsa.encrypt(message)
        json_message = json.dumps(encrypted_message)
        self.client.send_message(json_message)
        self.input_text.delete(0, tk.END)

    def receive_messages(self):
        while True:
            message = self.client.receive_message()
            decrypted_message = self.rsa.decrypt(message)
            self.output_text.insert(tk.END, "Received message: " + decrypted_message + "\n")
            self.output_text.see(tk.END)

    def run(self):
        self.window.mainloop()


if __name__ == "__main__":
    app = App('127.0.0.1', 8001)
    app.run()


ConnectionRefusedError: [WinError 10061] No connection could be made because the target machine actively refused it

Original message: Muhammad Ibrahim 10857
Encrypted message: [77, 117, 104, 80, 109, 109, 80, 185, 15, 73, 115, 199, 80, 104, 105, 109, 15, 134, 133, 192, 53, 191]
Decrypted message: Muhammad Ibrahim 10857
Public key: (17, 221)
Private key: (113, 221)
